# **Важно!**

Домашнее задание состоит из нескольких задач, которые вам нужно решить.
*   Баллы выставляются по принципу выполнено/невыполнено.
*   За каждую выполненую задачу вы получаете баллы (количество баллов за задание указано в скобках).

**Инструкция выполнения:** Выполните задания в этом же ноутбуке (места под решения **КАЖДОЙ** задачи обозначены как **#НАЧАЛО ВАШЕГО РЕШЕНИЯ** и **#КОНЕЦ ВАШЕГО РЕШЕНИЯ**)

**Как отправить задание на проверку:** Вам необходимо сохранить ваше решение в данном блокноте и отправить итоговый **файл .IPYNB** на учебной платформе в **стандартную форму сдачи домашнего задания.**

**Срок проверки преподавателем:** домашнее задание проверяется **в течение 3 дней после дедлайна сдачи** с предоставлением обратной связи

# **Прежде чем проверять задания:**

1. Перезапустите **ядро (restart the kernel)**: в меню, выбрать **Ядро (Kernel)**
→ **Перезапустить (Restart)**
2. Затем **Выполнить** **все ячейки (run all cells)**: в меню, выбрать **Ячейка (Cell)**
→ **Запустить все (Run All)**.

После ячеек с заданием следуют ячейки с проверкой **с помощью assert.**

Если в коде есть ошибки, assert выведет уведомление об ошибке.

Если в коде нет ошибок, assert отработает без вывода дополнительной информации.

---

In [ ]:
import math
import re
from random import randrange, shuffle, random, randint
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
"""
Задание 1 (1 балл): Генерация пакетов данных для обучения модели BERT

**Цель задания:** Написать код, который генерирует пакеты данных для обучения модели BERT с использованием Python и библиотеки NumPy.

**Задачи:**
1. Сформировать пустой список `batch` для хранения пакетов данных.
2. Инициализировать счетчики `positive` и `negative` для отслеживания положительных и отрицательных пар предложений.
3. В цикле, пока количество положительных и отрицательных пар в `batch` не достигнет заданного размера (`batch_size/2`), выполнить следующие шаги:
    - Выбрать случайные индексы двух предложений из списка `sentences`.
    - Получить сами предложения по выбранным индексам и сохранить их в переменные `tokens_a` и `tokens_b`.
    - Собрать входные данные для модели BERT, добавив специальные токены `[CLS]` и `[SEP]`. Используйте переменные `input_ids` и `segment_ids`
    для этой цели.
    - Реализовать маскирование токенов (MASK LM):
        - Определить количество токенов, которые будут маскированы (подставлены вместо них `[MASK]`).
        - Создать список кандидатов для маскирования (`cand_maked_pos`) и перемешать его.
        - Заполнить списки `masked_tokens` и `masked_pos` маскированными токенами и их позициями.
        - С вероятностью 80%, замаскировать токен, а с вероятностью 10%, заменить его случайным словом из словаря.
    - Дополнить последовательности нулями (`0`) до максимальной длины (`maxlen`).
    - Дополнить нулями маскированные токены и их позиции, чтобы их количество соответствовало максимально возможному (`max_pred`).
    - Определить, являются ли два предложения последовательными (`IsNext`) или нет (`NotNext`) на основе индексов предложений и счетчиков `positive`
    и `negative`.
    - Добавить полученные данные в список `batch` как положительную пару (`IsNext`) или отрицательную пару (`NotNext`).
    - Увеличить соответствующий счетчик (`positive` или `negative`) на `1`.
4. Вернуть сформированный список `batch` как результат выполнения функции.

**Примечания:**
- Задания выполняются в рамках задачи Next Sentence Prediction (NSP) для обучения модели BERT.
- Вы можете использовать функции и библиотеки Python, такие как `randrange`, `shuffle`, `random`, `randint`, `extend` и другие, для реализации
задачи.
- Весь необходимый функционал для работы с данными и списками будет предоставлен вам. Ваша задача - реализовать собственно формирование
пакетов данных на основе предоставленного кода и комментариев.
"""

def make_batch():
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    batch = []
    positive = 0
    negative = 0
    while positive < batch_size/2 or negative < batch_size/2:
        idx1, idx2 = randrange(len(sentences)), randrange(len(sentences))
        tokens_a, tokens_b = token_list[idx1], token_list[idx2]
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        n_pred = min(max_pred, max(1, int(round(len(input_ids) * 0.15))))
        cand_maked_pos = [
            i for i, token in enumerate(input_ids) if token != word_dict['[CLS]'] and
            token != word_dict['[SEP]']
        ]
        shuffle(cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random() < 0.8:
                input_ids[pos] = word_dict['[MASK]']
            elif random() < 0.5:
                index = randint(0, vocab_size-1)
                input_ids[pos] = word_dict[number_dict[index]]

        n_pad = maxlen - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if idx1 + 1 == idx2 and positive < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True])
            positive += 1
        elif idx1 + 1 != idx2 and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False])
            negative += 1

    return batch
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
def get_attn_pad_mask(seq_q, seq_k):
    """
    Функция для создания маски внимания, которая скрывает внимание на токенах-заполнителях (PAD).

    Args:
    - seq_q: Тензор с представлением запросов (batch_size x len_q).
    - seq_k: Тензор с представлением ключей (batch_size x len_k).

    Returns:
    - pad_attn_mask: Маска для внимания, скрывающая PAD-токены (batch_size x 1 x len_k(=len_q)).
    """
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()

    # Создаем маску, в которой значение True (1) будет соответствовать PAD-токенам.
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), где 1 означает маскирование

    # Расширяем маску до размерности batch_size x len_q x len_k, чтобы ее можно было использовать для маскирования внимания.
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k


def gelu(x):
    """
    Реализация активационной функции GELU (Gaussian Error Linear Unit) от Hugging Face.

    Args:
    - x: Входной тензор.

    Returns:
    - Результат применения функции GELU к входному тензору.
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

In [ ]:
class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # Токенное встраивание
        self.pos_embed = nn.Embedding(maxlen, d_model)  # Встраивание позиции
        self.seg_embed = nn.Embedding(n_segments, d_model)  # Встраивание сегмента (типа токена)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x, seg):
        seq_len = x.size(1)
        pos = torch.arange(seq_len, dtype=torch.long)
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        # Общее встраивание, включая токены, позиции и сегменты
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding)

In [ ]:
"""
## Задание 2 (1 балл): Реализация класса ScaledDotProductAttention

**Цель задания:** Написать класс `ScaledDotProductAttention`, который реализует механизм внимания с масштабированным скалярным произведением.

**Задачи:**
1. Создать класс `ScaledDotProductAttention`, который наследует `nn.Module` из библиотеки PyTorch.
2. В методе `forward` класса `ScaledDotProductAttention`, выполнить следующие действия:
   - Вычислите оценки (scores) как скалярное произведение матрицы запросов (Q) и транспонированной матрицы ключей (K).
   Масштабируйте оценки на обратный квадратный корень от размерности `d_k`.
   - Примените маску внимания (`attn_mask`) для скрытия определенных значений в оценках (scores).
   Для этого используйте метод `masked_fill_`, который заменяет элементы тензора на заданное значение, где маска равна 1.
   - Примените функцию softmax для вычисления весов внимания, которые определяют, насколько каждый элемент входных данных важен.
   - Вычислите контекст путем взвешенной суммы значений (V) с использованием весов внимания.
   - Верните контекст и веса внимания.
"""

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        # НАЧАЛО ВАШЕГО РЕШЕНИЯ
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        scores.masked_fill_(attn_mask, -1e9)
        weights = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(weights, V)

        return context, weights
        # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
## Задание 3 (1 балл): Реализация класса MultiHeadAttention

**Цель задания:** Написать класс `MultiHeadAttention`, который реализует многоголовое внимание (Multi-Head Attention) в модели трансформера.

**Задачи:**
1. Создать класс `MultiHeadAttention`, который наследует `nn.Module` из библиотеки PyTorch.
2. В конструкторе класса `MultiHeadAttention`, определить следующие атрибуты:
   - `W_Q`: Используйте линейный слой `nn.Linear` для создания преобразования запросов (Q).
   Размерность выхода должна быть `d_k * n_heads`. Прокомментируйте это как "Преобразование запросов".
   - `W_K`: Используйте линейный слой `nn.Linear` для создания преобразования ключей (K).
   Размерность выхода должна быть `d_k * n_heads`. Прокомментируйте это как "Преобразование ключей".
   - `W_V`: Используйте линейный слой `nn.Linear` для создания преобразования значений (V).
   Размерность выхода должна быть `d_v * n_heads`. Прокомментируйте это как "Преобразование значений".
3. В методе `forward` класса `MultiHeadAttention`, выполнить следующие действия:
   - Сохраните оригинальные запросы `Q` и определите размер пакета (batch_size).
   - Преобразуйте запросы, ключи и значения (`Q`, `K`, `V`) с использованием линейных преобразований `W_Q`, `W_K` и `W_V`,
   учитывая многие головы. Результаты преобразований должны быть разделены на головы и транспонированы.
   - Создайте маску внимания (`attn_mask`) с учетом входной маски `attn_mask` и повторите ее для каждой головы.
   - Примените механизм внимания (`ScaledDotProductAttention`) к головам, передав преобразованные запросы, ключи, значения и маску внимания.
   - Транспонируйте и объедините результаты голов для получения контекста.
   - Примените линейное преобразование и слой нормализации к контексту.
   - Верните нормализованный контекст и веса внимания.
"""

class MultiHeadAttention(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k*n_heads)   # Преобразование запросов
        self.W_K = nn.Linear(d_model, d_k*n_heads)   # Преобразование ключей
        self.W_V = nn.Linear(d_model, d_v*n_heads)   # Преобразование значений

    def forward(self, Q, K, V, attn_mask):
        residual, batch_size = Q, Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)

        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads*d_v)
        output = nn.Linear(n_heads*d_v, d_model)(context)

        return nn.LayerNorm(d_model)(output+residual), attn
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
## Задание 4 (1 балл): Реализация класса PoswiseFeedForwardNet

**Цель задания:** Написать класс `PoswiseFeedForwardNet`, который реализует полносвязную нейронную сеть с прямым распространением
(Feed Forward Neural Network) в модели трансформера.

**Задачи:**
1. Создать класс `PoswiseFeedForwardNet`, который наследует `nn.Module` из библиотеки PyTorch.
2. В конструкторе класса `PoswiseFeedForwardNet`, определить два атрибута:
   - `fc1`: Используйте линейный слой `nn.Linear`, чтобы создать первый линейный слой прямого распространения (feed-forward).
   Размерность входа должна быть `d_model`, а размерность выхода - `d_ff`. Прокомментируйте это как "Первый линейный слой (прямого распространения)".
   - `fc2`: Используйте линейный слой `nn.Linear`, чтобы создать второй линейный слой прямого распространения.
   Размерность входа должна быть `d_ff`, а размерность выхода - `d_model`. Прокомментируйте это как "Второй линейный слой (прямого распространения)".
3. В методе `forward` класса `PoswiseFeedForwardNet`, выполнить следующие действия:
   - Пропустите входные данные `x` через первый линейный слой `fc1`.
   - Примените функцию активации GELU (Gaussian Error Linear Unit) к результату первого линейного слоя.
   - Пропустите результат через второй линейный слой `fc2`.
   - Верните полученный результат в качестве выходных данных модели.
"""


class PoswiseFeedForwardNet(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)   # Первый линейный слой (прямого распространения)
        self.fc2 = nn.Linear(d_ff, d_model)   # Второй линейный слой (прямого распространения)

    def forward(self, x):
        x = self.fc1(x)
        output = self.fc2(gelu(x))

        return output
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
## Задание 5 (1 балл): Реализация класса EncoderLayer

**Цель задания:** Написать класс `EncoderLayer`, который представляет собой один слой кодировщика в модели трансформера.

**Задачи:**
1. Создать класс `EncoderLayer`, который наследует `nn.Module` из библиотеки PyTorch.
2. В конструкторе класса `EncoderLayer`, определить два атрибута:
   - `enc_self_attn`: Используйте класс `MultiHeadAttention`, чтобы создать многоголовое внимание для кодировщика.
   Прокомментируйте это как "Многоголовое внимание для кодировщика".
   - `pos_ffn`: Используйте класс `PoswiseFeedForwardNet`, чтобы создать полносвязную нейронную сеть с прямым распространением.
   Прокомментируйте это как "Полносвязная нейронная сеть с прямым распространением".
3. В методе `forward` класса `EncoderLayer`, выполнить следующие действия:
   - Применить многоголовое внимание `enc_self_attn` для самокодирования кодировщика, используя входные данные
   `enc_inputs` в роли запросов, ключей и значений.
   - Применить полносвязную нейронную сеть с прямым распространением `pos_ffn` для обработки выходных данных многоголового внимания.
   - Вернуть обработанные данные и веса внимания.
"""

class EncoderLayer(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()   # Многоголовое внимание для кодировщика
        self.pos_ffn = PoswiseFeedForwardNet()   # Полносвязная нейронная сеть с прямым распространением

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        enc_outputs = self.pos_ffn(enc_outputs)

        return enc_outputs, attn
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
"""
**Задание 6 (1 балл): Реализация модели BERT**

**Цель:** Ваша задача - реализовать модель BERT (Bidirectional Encoder Representations from Transformers) с использованием PyTorch.

**Инструкции:**

1. Ваша задача - реализовать класс BERT.

Описание задания для студентов: Реализация модели BERT

1. Создайте класс `BERT`, который будет наследоваться от `nn.Module`. В этом классе вы должны реализовать архитектуру BERT.

2. В конструкторе класса `BERT`, выполните следующие шаги:
   - Создайте модуль встраивания (Embedding) и добавьте его в класс.
   - Создайте список слоев кодировщика (EncoderLayer) и добавьте его в класс. Количество слоев кодировщика (`n_layers`) можно задать в качестве аргумента конструктора.
   - Создайте линейный слой (`nn.Linear`) для извлечения признаков из первого токена (CLS) и добавьте функцию активации (гиперболический тангенс).
   - Создайте второй линейный слой для извлечения скрытых признаков из выхода трансформера на позициях маскированных токенов. В качестве функции активации используйте GELU (Gaussian Error Linear Unit).
   - Добавьте слой нормализации (LayerNorm) и линейный классификатор для задачи классификации.

3. Реализуйте метод `forward`, который принимает входные данные (`input_ids`, `segment_ids`, `masked_pos`) и выполняет следующие действия:
   - Применяет входные данные через встраивание и кодировщик.
   - Извлекает признаки из первого токена (CLS) для классификации.
   - Извлекает скрытые признаки из выхода трансформера на позициях маскированных токенов.
   - Возвращает результаты классификации и предсказания маскированных токенов.

4. Обратите внимание на использование функций активации, слоев нормализации и линейных слоев в методе `forward`.

5. Создайте декодер, который будет использоваться совместно с встраиванием для задачи предсказания маскированных токенов.

6. Установите веса декодера равными весам встраивания.

7. Возвращайте результаты предсказания маскированных токенов и классификации.
"""


class BERT(nn.Module):
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.linear1 = nn.Linear(d_model, d_model)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(d_model, d_model)
        self.gelu = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        embedded = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(embedded, enc_self_attn_mask)

        h_pooled = self.tanh(self.linear1(output[:, 0]))
        logits_clf = self.classifier(h_pooled)

        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1))
        h_masked = torch.gather(output, 1, masked_pos)
        h_masked = self.norm(self.gelu(self.linear2(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias

        return logits_lm, logits_clf
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [ ]:
# Параметры BERT
maxlen = 30  # максимальная длина
batch_size = 6
max_pred = 5  # максимальное количество предсказываемых токенов
n_layers = 6  # количество слоев в кодировщике
n_heads = 12  # количество голов в Multi-Head Attention
d_model = 768  # размер вложения
d_ff = 768 * 4  # 4*d_model, размер скрытого слоя FeedForward
d_k = d_v = 64  # размерности K(=Q), V
n_segments = 2

text = (
    'Привет, как дела? Я - Ромео.\n'
    'Привет, Ромео. Меня зовут Джульетта. Приятно познакомиться.\n'
    'Приятно познакомиться. Как твои дела сегодня?\n'
    'Здорово. Моя бейсбольная команда выиграла соревнование.\n'
    'Поздравляю, Джульетта.\n'
    'Спасибо, Ромео'
)
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # фильтруем '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))
word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
number_dict = {i: w for i, w in enumerate(word_dict)}
vocab_size = len(word_dict)

token_list = list()
for sentence in sentences:
    arr = [word_dict[s] for s in sentence.split()]
    token_list.append(arr)

model = BERT()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch = make_batch()
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

for epoch in range(100):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens)  # для маскированной модели языка
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext)  # для классификации предложений
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Эпоха:', '%04d' % (epoch + 1), 'функция потерь =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

# Предсказание маскированных токенов и isNext
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('Список маскированных токенов: ', [pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('Предсказанный список маскированных токенов: ', [pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('Предсказано isNext : ', True if logits_clsf else False)

Эпоха: 0010 функция потерь = 3.898372
Эпоха: 0020 функция потерь = 2.532009
Эпоха: 0030 функция потерь = 2.519258
Эпоха: 0040 функция потерь = 1.953122
Эпоха: 0050 функция потерь = 1.161394
Эпоха: 0060 функция потерь = 1.530854
Эпоха: 0070 функция потерь = 1.116969
Эпоха: 0080 функция потерь = 0.901022
Эпоха: 0090 функция потерь = 1.048926
Эпоха: 0100 функция потерь = 0.817799
Привет, как дела? Я - Ромео.
Привет, Ромео. Меня зовут Джульетта. Приятно познакомиться.
Приятно познакомиться. Как твои дела сегодня?
Здорово. Моя бейсбольная команда выиграла соревнование.
Поздравляю, Джульетта.
Спасибо, Ромео
['[CLS]', 'приятно', 'познакомиться', 'как', 'твои', 'дела', 'сегодня', '[SEP]', '[MASK]', 'ромео', 'меня', 'зовут', 'джульетта', '[MASK]', 'познакомиться', '[SEP]']
Список маскированных токенов:  [7, 10]
Предсказанный список маскированных токенов:  [7, 10]
isNext :  False
Предсказано isNext :  False


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.8 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

In [ ]:
"""
**Задание 7 (3 балла): Реализация модели BERT из библиотеки transformers**

**Цель:** Ваша задача - реализовать модель BERT с использованием библиотеки transformers на наборе данных 20newsgroups.
Вывести на печать метрики модели на тестовом наборе test_data с помощью classification_report
"""

# Загрузка предварительно обученного BERT
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
# Импорт необходимых библиотек
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.metrics import accuracy_score
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import random
# Загрузка датасета "20 Newsgroups"
from sklearn.datasets import fetch_20newsgroups
def read_20newsgroups(test_size=0.2):

    dataset = fetch_20newsgroups(subset="all", shuffle=True, remove=("headers", "footers", "quotes"))
    documents = dataset.data
    labels = dataset.target

    return train_test_split(documents, labels, test_size=test_size), dataset.target_names


(train_texts, valid_texts, train_labels, valid_labels), target_names = read_20newsgroups()

In [ ]:
model_name = "bert-base-uncased"
max_length = 512

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.empty_cache()
!nvidia-smi

cuda
Mon Oct 30 04:11:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [ ]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds))
    return {
        'accuracy': acc,
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    logging_steps=400,
    save_steps=400,
    evaluation_strategy="steps",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
400,0.959600,1.094606,0.676923
800,1.045100,0.992490,0.694430
1200,0.732100,0.934562,0.727851
1600,0.617600,0.881277,0.738992


              precision    recall  f1-score   support

           0       0.40      0.39      0.40       151
           1       0.75      0.57      0.65       202
           2       0.40      0.73      0.51       195
           3       0.50      0.57      0.53       183
           4       0.70      0.56      0.62       205
           5       0.75      0.81      0.78       215
           6       0.76      0.71      0.73       193
           7       0.84      0.69      0.76       196
           8       0.75      0.70      0.72       168
           9       0.87      0.88      0.88       211
          10       0.91      0.82      0.86       198
          11       0.77      0.72      0.74       201
          12       0.61      0.61      0.61       202
          13       0.82      0.84      0.83       194
          14       0.83      0.77      0.80       189
          15       0.59      0.88      0.71       202
          16       0.78      0.52      0.63       188
          17       0.84    

TrainOutput(global_step=1886, training_loss=0.795119640556651, metrics={'train_runtime': 3336.8111, 'train_samples_per_second': 9.036, 'train_steps_per_second': 0.565, 'total_flos': 7934606683373568.0, 'train_loss': 0.795119640556651, 'epoch': 2.0})

In [ ]:
trainer.evaluate()
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

              precision    recall  f1-score   support

           0       0.50      0.63      0.56       151
           1       0.78      0.68      0.73       202
           2       0.69      0.67      0.68       195
           3       0.64      0.64      0.64       183
           4       0.82      0.74      0.78       205
           5       0.91      0.82      0.86       215
           6       0.83      0.78      0.80       193
           7       0.84      0.69      0.76       196
           8       0.61      0.79      0.69       168
           9       0.93      0.86      0.89       211
          10       0.98      0.85      0.91       198
          11       0.77      0.79      0.78       201
          12       0.62      0.76      0.68       202
          13       0.84      0.87      0.86       194
          14       0.55      0.86      0.67       189
          15       0.73      0.81      0.77       202
          16       0.73      0.73      0.73       188
          17       0.89    

{'eval_loss': 0.8812772035598755,
 'eval_accuracy': 0.7389920424403184,
 'eval_runtime': 129.7095,
 'eval_samples_per_second': 29.065,
 'eval_steps_per_second': 1.819,
 'epoch': 2.0}

In [ ]:
"""
**Задание 8 (3 балла): Реализация модели BART из библиотеки transformers**

**Цель:** Ваша задача - реализовать модель BART с использованием библиотеки transformers на наборе данных 20newsgroups.
Вывести на печать метрики модели на тестовом наборе test_data с помощью classification_report
"""
from transformers import BartTokenizerFast, BartForSequenceClassification
# Загрузка предварительно обученного BART
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
model_name = "facebook/bart-base"
max_length = 512
tokenizer = BartTokenizerFast.from_pretrained(model_name, do_lower_case=True)

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)

In [ ]:
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)

In [ ]:
model = BartForSequenceClassification.from_pretrained(model_name, num_labels=len(target_names)).to(device)

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['classification_head.out_proj.weight', 'classification_head.out_proj.bias', 'classification_head.dense.weight', 'classification_head.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions[0].argmax(-1)
    acc = accuracy_score(labels, preds)
    print(classification_report(labels, preds))
    return {
        'accuracy': acc,
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    load_best_model_at_end=True,
    logging_steps=400,
    save_steps=400,
    evaluation_strategy="steps",
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
400,2.176900,1.220463,0.623607
800,1.241100,1.105368,0.669496
1200,1.144600,1.046774,0.697878
1600,1.068100,0.965019,0.708488
2000,0.924000,0.958349,0.711671
2400,0.749400,0.932397,0.732891
2800,0.764900,0.855966,0.747215
3200,0.709600,0.861367,0.748541
3600,0.663900,0.852308,0.754111


              precision    recall  f1-score   support

           0       0.48      0.52      0.50       149
           1       0.55      0.61      0.58       188
           2       0.45      0.42      0.44       173
           3       0.64      0.04      0.07       190
           4       0.30      0.77      0.43       195
           5       0.74      0.70      0.72       192
           6       0.72      0.77      0.74       195
           7       0.45      0.81      0.58       172
           8       0.82      0.60      0.69       203
           9       0.95      0.80      0.87       200
          10       0.91      0.90      0.91       219
          11       0.66      0.70      0.68       199
          12       0.48      0.33      0.39       208
          13       0.95      0.67      0.78       209
          14       0.90      0.69      0.78       198
          15       0.75      0.76      0.75       211
          16       0.65      0.65      0.65       198
          17       0.85    

TrainOutput(global_step=3770, training_loss=1.032909423666228, metrics={'train_runtime': 5628.7399, 'train_samples_per_second': 5.357, 'train_steps_per_second': 0.67, 'total_flos': 9248516178935808.0, 'train_loss': 1.032909423666228, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

              precision    recall  f1-score   support

           0       0.60      0.56      0.58       149
           1       0.71      0.70      0.70       188
           2       0.66      0.71      0.68       173
           3       0.70      0.71      0.70       190
           4       0.81      0.71      0.76       195
           5       0.83      0.83      0.83       192
           6       0.79      0.86      0.82       195
           7       0.58      0.81      0.67       172
           8       0.76      0.81      0.78       203
           9       0.78      0.89      0.83       200
          10       0.92      0.93      0.92       219
          11       0.83      0.79      0.81       199
          12       0.77      0.60      0.67       208
          13       0.87      0.89      0.88       209
          14       0.80      0.81      0.81       198
          15       0.73      0.85      0.78       211
          16       0.70      0.71      0.71       198
          17       0.79    

{'eval_loss': 0.8523082733154297,
 'eval_accuracy': 0.7541114058355438,
 'eval_runtime': 202.9059,
 'eval_samples_per_second': 18.58,
 'eval_steps_per_second': 4.647,
 'epoch': 2.0}